# 3-Phase bbox3 Optimization with Fix Direction

Implementing the yongsukprasertsuk approach:
1. Phase A: Short runs (2min) with multiple n,r combinations
2. Phase B: Medium runs (10min) on top candidates
3. Phase C: Long runs (20min) on best few

Key techniques:
- fix_direction: Rotation tightening using convex hull
- repair_overlaps: Replace invalid configs with donor solution

In [1]:
import os
import shutil
import subprocess
import time
import json
import numpy as np
import pandas as pd
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
from shapely.strtree import STRtree

# Setup paths
WORK_DIR = '/home/code/experiments/005_3phase_bbox3'
BBOX3_BIN = '/home/code/code/bbox3'
SUBMISSION = '/home/submission/submission.csv'
BASELINE = '/home/code/experiments/003_better_baseline/submission.csv'

os.makedirs(WORK_DIR, exist_ok=True)

# Tree shape
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

print("Setup complete")

Setup complete


In [2]:
def get_tree_polygon(cx, cy, deg):
    """Create tree polygon at given position and rotation."""
    coords = list(zip(TX, TY))
    poly = Polygon(coords)
    poly = affinity.rotate(poly, deg, origin=(0, 0))
    poly = affinity.translate(poly, xoff=cx, yoff=cy)
    return poly

def has_overlap(polys):
    """Check if any polygons overlap."""
    if len(polys) <= 1:
        return False
    tree = STRtree(polys)
    for i, poly in enumerate(polys):
        indices = tree.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polys[idx]) and not poly.touches(polys[idx]):
                return True
    return False

def get_side_length(polys):
    """Get bounding box side length."""
    union = unary_union(polys)
    bounds = union.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def calc_score(df):
    """Calculate total score from dataframe."""
    total = 0
    for n in range(1, 201):
        group = df[df['n'] == n]
        if len(group) == 0:
            continue
        polys = [get_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) for _, row in group.iterrows()]
        side = get_side_length(polys)
        total += side**2 / n
    return total

def load_submission(path):
    """Load submission and parse values."""
    df = pd.read_csv(path)
    df['x_val'] = df['x'].astype(str).str.replace('s', '').astype(float)
    df['y_val'] = df['y'].astype(str).str.replace('s', '').astype(float)
    df['deg_val'] = df['deg'].astype(str).str.replace('s', '').astype(float)
    df['n'] = df['id'].apply(lambda x: int(str(x).split('_')[0]))
    return df

print("Helper functions defined")

Helper functions defined


In [3]:
def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side at given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def fix_direction_for_n(df, n, epsilon=1e-7):
    """Apply rotation tightening for a single N value."""
    group = df[df['n'] == n].copy()
    if len(group) == 0:
        return df, 0
    
    # Get all polygon points
    all_points = []
    for _, row in group.iterrows():
        poly = get_tree_polygon(row['x_val'], row['y_val'], row['deg_val'])
        all_points.extend(list(poly.exterior.coords))
    
    points_np = np.array(all_points)
    
    # Use convex hull for efficiency
    try:
        hull_points = points_np[ConvexHull(points_np).vertices]
    except:
        return df, 0
    
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    # Find optimal rotation
    res = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0.001, 89.999),
        method='bounded'
    )
    
    found_angle = float(res.x)
    found_side = float(res.fun)
    improvement = initial_side - found_side
    
    if improvement > epsilon:
        # Apply rotation to all trees in this group
        # Calculate rotation center
        cx = (points_np[:, 0].min() + points_np[:, 0].max()) / 2
        cy = (points_np[:, 1].min() + points_np[:, 1].max()) / 2
        
        angle_rad = np.radians(found_angle)
        c, s = np.cos(angle_rad), np.sin(angle_rad)
        
        for idx in group.index:
            x, y = df.loc[idx, 'x_val'], df.loc[idx, 'y_val']
            # Rotate position around center
            dx, dy = x - cx, y - cy
            new_x = dx * c - dy * s + cx
            new_y = dx * s + dy * c + cy
            df.loc[idx, 'x_val'] = new_x
            df.loc[idx, 'y_val'] = new_y
            df.loc[idx, 'deg_val'] += found_angle
        
        return df, improvement
    
    return df, 0

def fix_direction_all(df):
    """Apply rotation tightening to all N values."""
    total_improvement = 0
    for n in range(1, 201):
        df, imp = fix_direction_for_n(df, n)
        if imp > 0:
            total_improvement += imp
    return df, total_improvement

print("Fix direction functions defined")

Fix direction functions defined


In [4]:
def save_submission(df, path):
    """Save submission in correct format."""
    rows = []
    for _, row in df.iterrows():
        rows.append({
            'id': row['id'],
            'x': f"s{row['x_val']:.15f}",
            'y': f"s{row['y_val']:.15f}",
            'deg': f"s{row['deg_val']:.15f}"
        })
    pd.DataFrame(rows).to_csv(path, index=False)

def validate_submission(df):
    """Check for overlaps in all N groups."""
    failed_n = []
    for n in range(1, 201):
        group = df[df['n'] == n]
        if len(group) == 0:
            continue
        polys = [get_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) for _, row in group.iterrows()]
        if has_overlap(polys):
            failed_n.append(n)
    return failed_n

def repair_overlaps(df, donor_df, failed_n):
    """Replace overlapping N configs with donor solution."""
    for n in failed_n:
        # Remove current N rows
        df = df[df['n'] != n]
        # Add donor N rows
        donor_group = donor_df[donor_df['n'] == n].copy()
        df = pd.concat([df, donor_group], ignore_index=True)
    return df.sort_values('id').reset_index(drop=True)

print("Save/validate/repair functions defined")

Save/validate/repair functions defined


In [5]:
# Load current best and baseline
print("Loading submissions...")
current_df = load_submission(SUBMISSION)
baseline_df = load_submission(BASELINE)

current_score = calc_score(current_df)
print(f"Current score: {current_score:.6f}")
print(f"Target: 68.894234")
print(f"Gap: {current_score - 68.894234:.6f}")

Loading submissions...


Current score: 70.627569
Target: 68.894234
Gap: 1.733335


In [6]:
# Apply fix_direction to current submission
print("\nApplying fix_direction...")
fixed_df, total_imp = fix_direction_all(current_df.copy())

if total_imp > 0:
    print(f"Total side improvement from fix_direction: {total_imp:.6f}")
    
    # Validate
    failed_n = validate_submission(fixed_df)
    if failed_n:
        print(f"Overlaps detected in N: {failed_n}")
        print("Repairing with baseline...")
        fixed_df = repair_overlaps(fixed_df, baseline_df, failed_n)
    
    fixed_score = calc_score(fixed_df)
    print(f"Score after fix_direction: {fixed_score:.6f}")
    print(f"Score improvement: {current_score - fixed_score:.6f}")
    
    if fixed_score < current_score:
        save_submission(fixed_df, SUBMISSION)
        print("Saved improved submission")
        current_df = fixed_df
        current_score = fixed_score
else:
    print("No improvement from fix_direction")


Applying fix_direction...


No improvement from fix_direction


In [7]:
# Run bbox3 with different parameters
def run_bbox3(n_val, r_val, timeout=120):
    """Run bbox3 optimizer."""
    # Copy current submission to working directory
    shutil.copy(SUBMISSION, '/home/code/code/submission.csv')
    
    try:
        result = subprocess.run(
            [BBOX3_BIN, '-n', str(n_val), '-r', str(r_val)],
            capture_output=True,
            text=True,
            timeout=timeout,
            cwd='/home/code/code'
        )
        
        # Parse final score from output
        for line in result.stdout.split('\n'):
            if 'Final score' in line:
                try:
                    score = float(line.split(':')[-1].strip())
                    return score
                except:
                    pass
        return None
    except subprocess.TimeoutExpired:
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

print("bbox3 runner defined")

bbox3 runner defined


In [ ]:
# Phase A: Short runs to find promising parameters
print("\n=== PHASE A: Short runs (2 min each) ===")

phaseA_results = []
n_values = [1000, 1200, 1500, 1800, 2000]
r_values = [30, 60, 90]

for r in r_values:
    for n in n_values:
        print(f"Testing n={n}, r={r}...")
        score = run_bbox3(n, r, timeout=120)
        if score is not None:
            phaseA_results.append({'n': n, 'r': r, 'score': score})
            print(f"  Score: {score:.6f}")
            if score < current_score:
                print(f"  ** Improvement: {current_score - score:.6f}")
        else:
            print(f"  Timeout or error")

# Sort by score
phaseA_results.sort(key=lambda x: x['score'])
print(f"\nTop 5 Phase A results:")
for r in phaseA_results[:5]:
    print(f"  n={r['n']}, r={r['r']}: {r['score']:.6f}")

In [ ]:
# Check if any Phase A result improved
best_phaseA = phaseA_results[0] if phaseA_results else None

if best_phaseA and best_phaseA['score'] < current_score:
    print(f"\nBest Phase A result: n={best_phaseA['n']}, r={best_phaseA['r']}, score={best_phaseA['score']:.6f}")
    print(f"Improvement: {current_score - best_phaseA['score']:.6f}")
    
    # Load the improved submission
    improved_df = load_submission('/home/code/code/submission.csv')
    
    # Apply fix_direction
    print("Applying fix_direction...")
    fixed_df, _ = fix_direction_all(improved_df.copy())
    
    # Validate and repair
    failed_n = validate_submission(fixed_df)
    if failed_n:
        print(f"Repairing overlaps in N: {failed_n}")
        fixed_df = repair_overlaps(fixed_df, baseline_df, failed_n)
    
    final_score = calc_score(fixed_df)
    print(f"Final score after fix_direction: {final_score:.6f}")
    
    if final_score < current_score:
        save_submission(fixed_df, SUBMISSION)
        current_score = final_score
        print(f"Saved! New best: {current_score:.6f}")
else:
    print("\nNo improvement from Phase A")

In [ ]:
# Save metrics
metrics = {
    'cv_score': current_score,
    'target': 68.894234,
    'gap': current_score - 68.894234,
    'phaseA_results': phaseA_results[:5] if phaseA_results else [],
    'notes': '3-phase bbox3 optimization with fix_direction rotation tightening'
}

with open(f'{WORK_DIR}/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

# Copy submission to experiment folder
shutil.copy(SUBMISSION, f'{WORK_DIR}/submission.csv')

print(f"\n=== FINAL RESULTS ===")
print(f"Score: {current_score:.6f}")
print(f"Target: 68.894234")
print(f"Gap: {current_score - 68.894234:.6f}")